In [25]:
import os
import json

with open(
    os.path.join("data/2024-01-24", "90552.json"), encoding="utf-8"
) as f:
    data = json.load(f)

In [26]:
data

{'current': {'fromDateTime': '2024-01-24T17:47:17.301Z',
  'tillDateTime': '2024-01-24T18:47:17.301Z',
  'values': [{'name': 'PM1', 'value': 3.89},
   {'name': 'PM25', 'value': 6.12},
   {'name': 'PM10', 'value': 6.88},
   {'name': 'PRESSURE', 'value': 1014.36},
   {'name': 'HUMIDITY', 'value': 83.18},
   {'name': 'TEMPERATURE', 'value': 6.59}],
  'indexes': [{'name': 'AIRLY_CAQI',
    'value': 10.21,
    'level': 'VERY_LOW',
    'description': 'Great air here today!',
    'advice': 'Breathe deeply!',
    'color': '#6BC926'}],
  'standards': [{'name': 'WHO',
    'pollutant': 'PM10',
    'limit': 45.0,
    'percent': 15.29,
    'averaging': '24h'},
   {'name': 'WHO',
    'pollutant': 'PM25',
    'limit': 15.0,
    'percent': 40.82,
    'averaging': '24h'}]},
 'history': [{'fromDateTime': '2024-01-23T18:00:00.000Z',
   'tillDateTime': '2024-01-23T19:00:00.000Z',
   'values': [{'name': 'PM1', 'value': 8.99},
    {'name': 'PM25', 'value': 13.31},
    {'name': 'PM10', 'value': 14.93},
    {

In [27]:
import pandas as pd
import numpy as np
data = pd.DataFrame(data["history"])

In [28]:
from app.data_managers.namespaces import sensor_json_ns, column_names_ns
df = data.copy()
columns = [
    column
    for column in (
        sensor_json_ns.TILL_DATE_TIME,
        sensor_json_ns.INDEXES,
        sensor_json_ns.STANDARDS,
    )
    if column in df.columns
]
if columns:
    df = df.drop(columns=columns)

left_df = (
    df.drop(columns=sensor_json_ns.VALUES)
    if sensor_json_ns.VALUES in df.columns
    else df
)
right_df = df[sensor_json_ns.VALUES].apply(
    lambda values: pd.Series(values, dtype="object")
)

df = pd.concat(
    (left_df, right_df),
    axis=1,
)

columns = df.columns
result = pd.DataFrame()

for column in columns:
    series = df[column].apply(pd.Series)
    series = series.rename(columns={sensor_json_ns.VALUE: series.iloc[0, 0]})
    if sensor_json_ns.NAME in series.columns:
        mask = series[sensor_json_ns.NAME] != series.iloc[0, 0]
        series.iloc[mask] = np.nan
    result = pd.concat([result, series], axis=1)

columns = result.columns.to_list()
columns[0] = column_names_ns.TIME
result.columns = columns

if sensor_json_ns.NAME in result.columns:
    result = result.drop(columns=[sensor_json_ns.NAME])

if 0 in result.columns:
    result = result.drop(columns=[0])


KeyError: 'name'

In [23]:
series

,name,PM1
0,PM1,8.99
1,PM1,8.21
2,PM1,8.36
3,PM1,8.48
4,PM1,10.28
5,PM1,9.98
6,PM1,7.37
7,PM1,5.30
8,PM1,4.19
9,PM1,4.39
